In [1]:
%pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
import cv2
import numpy as np
import pickle

# Path to the 'chessboard_images' folder on your Desktop (VS Code workspace path)
data_path = 'chessboard_images'

# Chessboard dimensions (number of inner corners per chessboard row and column)
chessboard_size = (9, 6)  # Change this based on your chessboard size

# Prepare object points (3D points in real world space)
objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images
objpoints = []
imgpoints = []

# Read all images from the chessboard_images folder
for img_name in os.listdir(data_path):
    img_path = os.path.join(data_path, img_name)
    img = cv2.imread(img_path)
    
    if img is None:
        continue
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)
    
    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)
        
        # Draw and display the corners on the image
        cv2.drawChessboardCorners(img, chessboard_size, corners, ret)
        cv2.imshow('Chessboard', img)
        cv2.waitKey(500)  # Wait for 500ms to view the corner detection

cv2.destroyAllWindows()

# Camera calibration using the found corners
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# Save the calibration data to a file for future use
calibration_data = {
    'camera_matrix': mtx,
    'distortion_coefficients': dist,
    'rotation_vectors': rvecs,
    'translation_vectors': tvecs
}

with open('camera_calibration_data.pkl', 'wb') as f:
    pickle.dump(calibration_data, f)

print("Camera calibration complete and data saved.")


Camera calibration complete and data saved.


In [17]:
import cv2
import os
import numpy as np
import pickle

# Load the calibration data from the previous step
with open('camera_calibration_data.pkl', 'rb') as f:
    calibration_data = pickle.load(f)

camera_matrix = calibration_data['camera_matrix']
distortion_coefficients = calibration_data['distortion_coefficients']

# Initialize face recognizer (LBPH)
recognizer = cv2.face.LBPHFaceRecognizer_create()

# Path to the 'photoss' folder containing user images
data_path = '/Users/marwaelamrawy/Desktop/photoss'

# Load known face encodings (user images)
faces = []
labels = []
label_map = {}
current_label = 0

# Face detection cascade
haar_cascade_path = '/Users/marwaelamrawy/Desktop/haarcascade_frontalface_default.xml'  # Update this path if necessary
face_cascade = cv2.CascadeClassifier(haar_cascade_path)

# Load images from 'photoss' directory and train the recognizer
for user in os.listdir(data_path):
    user_path = os.path.join(data_path, user)
    if not os.path.isdir(user_path):
        continue
    label_map[current_label] = user
    for img_name in os.listdir(user_path):
        img_path = os.path.join(user_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Warning: Could not load image {img_path}")
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces_rect = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        if len(faces_rect) == 0:
            print(f"No faces detected in image: {img_path}")
            continue  # Skip this image if no faces are detected

        for (x, y, w, h) in faces_rect:
            face = gray[y:y+h, x:x+w]
            faces.append(face)
            labels.append(current_label)
    
    current_label += 1

# Ensure faces and labels have data before training
if len(faces) == 0 or len(labels) == 0:
    print("Error: No faces were detected in the images.")
else:
    # Train the face recognizer with the loaded faces and labels
    recognizer.train(faces, np.array(labels))
    print(f"Training complete. {len(faces)} faces used.")

# Open the camera for live feed
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Camera not found or unable to open.")
else:
    print("Camera started. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image from camera.")
        break

    # Undistort the frame using the calibration data
    frame = cv2.undistort(frame, camera_matrix, distortion_coefficients)

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the live feed
    faces_rect = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces_rect:
        face = gray[y:y+h, x:x+w]
        label, confidence = recognizer.predict(face)

        name = label_map.get(label, "Unknown")
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, f"{name} ({int(confidence)})", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Display the live frame
    cv2.imshow('User Authentication', frame)

    # Quit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Error: No faces were detected in the images.
Camera started. Press 'q' to quit.
Error: Failed to capture image from camera.
